In [ ]:
!conda create -n my_env python=3.7 -y

In [ ]:
!conda init

In [2]:
import os

# Set the environment variables
os.environ['PATH'] = '/opt/conda/bin:' + os.environ['PATH']
os.environ['CONDA_DEFAULT_ENV'] = 'my_env'  # Replace 'myenv' with your Conda environment name

# Restart the kernel to apply changes
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)


{'status': 'ok', 'restart': True}

In [ ]:
!export PATH="/opt/conda/envs/my_env/bin:$PATH"
!export PYTHONPATH="/opt/conda/envs/my_env/lib/python3.7/site-packages:$PYTHONPATH"

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import librosa
import os
import numpy as np

In [3]:
!pip install pickle5
!pip install omegaconf==2.0.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 2.7 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp310-cp310-linux_x86_64.whl size=125214 sha256=7dd615e4ae8499c8f70d853e781d67bf25308657ed59cd5995f47efbb9b5f7a7
  Stored in directory: /root/.cache/pip/wheels/7d/14/ef/4aab19d27fa8e58772be5c71c16add0426acf9e1f64353235c
Successfully built pickle5
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [ ]:
required = [
            "cffi",
            "cython",
            'dataclasses; python_version<"3.7"',
            "hydra-core>=1.0.7,<1.1",
            "omegaconf<2.1",
            'numpy<1.20.0; python_version<"3.7"',
            'numpy; python_version>="3.7"',
            "regex",
            "sacrebleu>=1.4.12",
            "torch",
            "tqdm",
            "bitarray",
            "torchaudio>=0.8.0",
        ]

In [ ]:
for r in required:
    os.system(f"pip install {r}")

In [ ]:
os.chdir("/kaggle/input/su-ass3")

In [ ]:
import trans

In [ ]:
class custom_dataset(Dataset):
    def __init__(self, path):
        real_path = os.path.join(path, 'real')
        fake_path = os.path.join(path, 'fake')
        real_files = os.listdir(real_path)
        fake_files = os.listdir(fake_path)
        label = []
        for i in range(len(real_files)):
            real_files[i] = os.path.join(real_path,real_files[i])
            label.append(1)
        for i in range(len(fake_files)):
            fake_files[i] = os.path.join(fake_path,fake_files[i])
            label.append(0)
        self.files = real_files+fake_files
        self.label = label
    
    def __len__(self):
        return len(self.fales)
    
    def __getitem__(self, idx):
        file = self.files[idx]
        label = self.label[idx]
        x,sr = librosa.load(file,sr=16000)
        x = pad(x)
        x = torch.from_numpy(x)
        return x, label

def pad(x, max_len=64600):
    x_len = x.shape[0]
    if x_len >= max_len:
        return x[:max_len]
    # need to pad
    num_repeats = int(max_len / x_len)+1
    padded_x = np.tile(x, (1, num_repeats))[:, :max_len][0]
    return padded_x	

In [ ]:
train_dataset = custom_dataset("/kaggle/input/su-ass3/for-2seconds/validation")
valid_dataset = custom_dataset("/kaggle/input/su-ass3/for-2seconds/testing")

In [ ]:
batch_size = 32
epochs = 50
lr = 1e-4

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=batch_size)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size)

In [ ]:
mod = trans.load_model()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
mod = mod.to(device)
opt = optim.Adam(mod.parameters(),lr=lr)
loss_fn = nn.CrossEntropyLoss()
min_loss= np.inf
for epoch in range(epochs):
    for X, Y in train_loader:
        X = X.float().to(device)
        Y = Y.to(device)
        opt.zero_grad()
        prob = mod(X)
        loss = loss_fn(prob,Y)
        loss.backward()
        opt.step()
    loss_ = []
    with torch.no_grad():
        for X, Y in valid_loader:
            X = X.float().to(device)
            Y = Y.to(device)
            prob = mod(X)
            loss = loss_fn(prob,Y)
            loss.backward()
            loss_.append(loss.item())
    if min_loss > np.mean(loss_):
        min_loss  = np.mean(loss_)
        torch.save(mod.state_dict(),"best_model.py")